In [332]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from currency_converter import CurrencyConverter
import datetime
from forex_python.converter import CurrencyRates

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [333]:
df = pd.read_csv('https://drive.google.com/uc?export=download&id=130KYOX8O4wrP_T8vdz2GfvJRQ03ONmE7', index_col=0)

#### 1. Удаление дубликатов и стобцов, которые могут быть не важны для исследования.  
   
Парковка - пропущено 13417 из 23368 - 57,4%  
Название ЖК - пропущено 17520 из 23368 - 74,9%  
Серия дома - пропущено 21205 из 23368 - 90,7%  
Высота потолков, м  - пропущено 12162 из 23368 - 52%   
Мусоропровод - пропущено 10522 из 23368 - 45%  

Тип, Телефоны, Ссылка на объявление, площадь комнат, описание - параметры, не влияющие на стоимость

In [334]:
# Удаляем дубликаты по ID 
df = df.drop_duplicates(subset=['ID  объявления'], keep='first').reset_index(drop = True)

In [335]:
df = df.drop(['Парковка', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Мусоропровод', 'Тип', 'Телефоны', 'Ссылка на объявление', 'Площадь комнат, м2', 'Описание'], axis = 1)

#### 2. Чистка данных  
  
* Работа с пропусками
* Фильтрация колонок
* Создание новых колонок
* Переименование колонок

In [336]:
# Кол-во комнат удаляем информацию про тип, оставляем только цифру, заполняем пропуски медианой
df['Количество комнат'] = df['Количество комнат'].apply(lambda x: int(x.split(', ')[0]) if isinstance(x, str) and ', ' in x else x)
df['Количество комнат']= df['Количество комнат'].fillna(df['Количество комнат'].median())
df.rename(columns={'Количество комнат': 'number_of_rooms'}, inplace=True)


In [337]:
#Признак метро делим на два, Метро и время в пути
df.insert(loc=3, column='travel_time', value=[1] * len(df))
df['Метро'] = df['Метро'].fillna(df['Метро'].mode()[0])

# Обработка столбца 'Метро', деление на два столбца: 'metro' и 'travel_time'
def split_metro(row):
    if isinstance(row, str) and ' (' in row:
        return row.split(' (')[1].replace(')', '')
    else:
        return np.nan

df['travel_time'] = df['Метро'].apply(split_metro)
df['Метро'] = df['Метро'].apply(lambda row: row.split(' (')[0] if isinstance(row, str) and ' (' in row else row)
df.rename(columns={'Метро': 'metro'}, inplace=True)


In [338]:
# Пропусков в адресах нет, просто переименовываем и удаляем все строки, в которых указан другой город
df.rename(columns={'Адрес': 'address'}, inplace=True)
df = df[df['address'].str.contains("Москва")].reset_index(drop = True)

In [339]:
# Обрабатываем колонку с площадью, берём только общую площадь
df['Площадь, м2'] = [row.split('/')[0] for row in df['Площадь, м2']]
df.rename(columns={'Площадь, м2': 'square_m2'}, inplace=True)

In [340]:
# Разбиваем колонку дом
df['floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df['floors'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

In [341]:
# Из информации о цене достаем стоимость за месяц + депозит, и удаляем старый столбец
df['rent_price'] = [(row.split('/')[0]) for row in df['Цена']]
# df['deposit'] = [(row.split(' - ')[1].split(',')[0]) for row in df['Цена']]
df['deposit'] = [row.split(' - ')[1].split(',')[0] if 'Залог' in row else '0' for row in df['Цена']]

df = df.drop(['Цена'], axis = 1)

In [342]:
# Конвертация в рубли и приведение к интовому значению

import json
import requests
import re

url = ("https://iss.moex.com/iss/engines/currency/markets/selt/securities.jsonp?"
       "iss.only=securities,marketdata&"
       "securities=CETS:USD000UTSTOM,CETS:EUR_RUB__TOM&"
       "lang=ru&iss.meta=off&iss.json=extended&callback=angular.callbacks._gk")
data = requests.get(url)

# Обрежем лишнее (вызов функции и переводы строк)
text = data.text[22:len(data.text)-1:]
text = re.sub(r'\n', "", text)

json_string = json.loads(text)
list_price = []
for ss in json_string[1]['securities']:
    list_price.append(ss['PREVWAPRICE'])

# Задаем курсы валют на сегодняшний день (для примера)
usd_to_rub_rate = list_price[1]  # курс доллара к рублю
eur_to_rub_rate = list_price[0]  # курс евро к рублю

def convert_price_rent(text):
    if text != 0:
        parts = text.split(' ')
        amount = float(parts[0])  # получаем количество и преобразуем в float
        currency = parts[1]  # получаем тип валюты
        
        if currency == 'руб.':
            result = amount
        elif currency == '$':
            result = (amount * usd_to_rub_rate)
        elif currency == '€':
            result = (amount * eur_to_rub_rate)
        return int(result)
    else:
        return int(text)
    
def convert_price_dep(text):
    if text == '0':
        result = 0
        return int(result)
    elif ('€' in text):
        text = text.replace('€', "")
        result = float(text) * eur_to_rub_rate
        return int(result)

    if len(text.split(' ')) > 1:    
        parts = text.split(' ')
        amount = float(parts[0])  # получаем количество и преобразуем в float
        currency = parts[1]  # получаем тип валюты
        
        if currency == 'руб.':
            result = amount
        elif currency == '$':
            result = (amount * usd_to_rub_rate)
        return int(result)  
    else:
        return int(text)  
    

# Применяем функцию convert_price к столбцам 'rent_price' и 'deposit' в DataFrame
df['rent_price'] = df['rent_price'].apply(convert_price_rent)
df['deposit'] = df['deposit'].apply(convert_price_dep)

In [343]:
# Заполняем пропуски в столбце ремонт и переименовываем
df['Ремонт']= df['Ремонт'].fillna(df['Ремонт'].mode()[0])
df.rename(columns={'Ремонт': 'renovation_type'}, inplace=True)

In [345]:
# Обрабатываем колонку "балкон"
df['Балкон'] = df['Балкон'].fillna('Балкон (0), Лоджия (0)')
df['Лоджия'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Лоджия' else '(0)' for row in df['Балкон']]
df['Балкон'] = [row.split(' ')[1] if row.split(' ')[0] == 'Балкон' else '(0)' for row in df['Балкон']]

In [346]:
# Корректируем значения, избавляясь от скобок и переделывая из str в int
df['Лоджия'] = [row.split('(')[1] for row in df['Лоджия']]
df['Лоджия'] = [int(row.split(')')[0]) for row in df['Лоджия']]
df['Балкон'] = [row.split('(')[1] for row in df['Балкон']]
df['Балкон'] = [int(row.split(')')[0]) for row in df['Балкон']]

In [349]:
# Переименовываем балкон и лоджия
df.rename(columns={'Балкон': 'balcony'}, inplace=True)
df.rename(columns={'Лоджия': 'loggia'}, inplace=True)

In [352]:
# Заполняем пропуски в окнах и переименовываем
df['Окна'] = df['Окна'].fillna(df['Окна'].mode()[0])
df.rename(columns={'Окна': 'window'}, inplace=True)

In [355]:
# Столбец Санузел разбиваем на два
df['Санузел'] = df['Санузел'].fillna(df['Санузел'].mode()[0])
df['Санузел совмещенный'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Совмещенный' else '(0)' for row in df['Санузел']]
df['Санузел раздельный'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Раздельный' else '(0)' for row in df['Санузел']]
df['Санузел совмещенный'] = [row.split('(')[1] for row in df['Санузел совмещенный']]
df['Санузел совмещенный'] = [int(row.split(')')[0]) for row in df['Санузел совмещенный']]
df['Санузел раздельный'] = [row.split('(')[1] for row in df['Санузел раздельный']]
df['Санузел раздельный'] = [int(row.split(')')[0]) for row in df['Санузел раздельный']]
df = df.drop(['Санузел'], axis = 1)

In [358]:
# Переименовываем
df.rename(columns={'Санузел совмещенный': 'сombined_bath'}, inplace=True)
df.rename(columns={'Санузел раздельный': 'separate_bath'}, inplace=True)

In [361]:
# Заполняем пропуски в разделе можно с детьми/животными
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna(df['Можно с детьми/животными'].mode()[0])
df.rename(columns={'Можно с детьми/животными': 'child_pets_allow'}, inplace=True)

In [364]:
# Избавляемся от NaN, если меньше 5 этажей - нет лифта, в остальном по одному каждого лифта
df['Лифт'] = [row if row == row else
              ['Пасс (1), Груз (1)' if df['floors'].iloc[i]>5 else 'Пасс (0), Груз (0)'][0]
              for i, row in enumerate(df['Лифт'])]

In [365]:
# Разбиваем колонку с лифтами на две раздельные 
df['Лифт пасс'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Пасс' else '(0)' for row in df['Лифт']]
df['Лифт груз'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Груз' else '(0)' for row in df['Лифт']]

In [366]:
# Убираем скобки
df['Лифт пасс'] = [row.split('(')[1] for row in df['Лифт пасс']]
df['Лифт пасс'] = [int(row.split(')')[0]) for row in df['Лифт пасс']]
df['Лифт груз'] = [row.split('(')[1] for row in df['Лифт груз']]
df['Лифт груз'] = [int(row.split(')')[0]) for row in df['Лифт груз']]
df = df.drop(['Лифт'], axis = 1)

In [372]:
# Переименовываем
df.rename(columns={'Лифт пасс': 'passenger_lift'}, inplace=True)
df.rename(columns={'Лифт груз': 'cargo_lift'}, inplace=True)

In [375]:
# Заполняем пропуски в столбце дополнительно
df['Дополнительно'] = df['Дополнительно'].fillna(df['Дополнительно'].mode()[0])

In [378]:
df.to_csv('data.csv')